In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Dict, Optional

import cfd_common as comm

NI=comm.NI
NJ=comm.NJ
NIM=NI-1

In [ ]:
@dataclass
class DomainInfo:
    L: float
    NI: int
    NJ: int
    IDIM: int
    JDIM: int
    IJDIM: int
    NIM: int
    NJM: int
    DX0: float
    DY0: float
    
def set_domain(NI, NJ, DX0, DY0):
    global domain

    IDIM = NI + 2
    JDIM = NJ + 2
    IJDIM = max(IDIM, JDIM)
    NIM   = NI - 1
    NJM   = NJ - 1

    domain = DomainInfo(
        NI=NI, NJ=NJ,
        IDIM=IDIM, JDIM=JDIM,
        IJDIM=IJDIM, NIM=NIM, NJM=NJM,
        DX0=DX0, DY0=DY0
    )

@dataclass
class BoundaryValues:
    W: float
    E: float
    S: float
    N: float

@dataclass
class FieldEquation:
    name: str
    # Quantity
    PHI: np.ndarray               
    # Source Term  
    SOR: np.ndarray
    # Coefficients
    AP: np.ndarray
    AW: np.ndarray
    AE: np.ndarray
    AS: np.ndarray
    AN: np.ndarray
    # Relaxation Factor
    relax: float
    # Boundary Cond
    BC: BoundaryValues
    # Neumann Option
    neumann: Optional[Dict[str, Optional[str]]] = None
    def __getitem__(self, idx):
        return self.PHI[idx]     # e.g., phi[i], phi[i,j]

    def __setitem__(self, idx, value):
        self.PHI[idx] = value   # e.g., phi[i,j] = value

In [ ]:
# T = FieldEquation(
#     name = "T",
#     phi = np.zeros((IDIM,JDIM)),
#     SOR = np.zeros((IDIM,JDIM)),
#     AP  = np.zeros((IDIM,JDIM)),
#     AW  = np.zeros((IDIM,JDIM)),
#     AE  = np.zeros((IDIM,JDIM)),
#     AS  = np.zeros((IDIM,JDIM)),
#     AN  = np.zeros((IDIM,JDIM)),
#     bc  = BoundaryValues(W=TW, E=TE, S=TS, N=TN),
#     neumann = {"W": None, "E": None, "S": "neumann", "N": None},
#     relax = 0.8,
# )


In [ ]:
def set_BC(PHI):
    for j in range(0, NJ): PHI[0][j]  =  PHI.BC.W  # West Wall
    for j in range(0, NJ): PHI[NI][j] =  PHI.BC.E  # East Wall
    for i in range(0, NI): PHI[i][0]  =  PHI.BC.S  # South Wall
    for i in range(0, NI): PHI[i][NJ] =  PHI.BC.N  # North Wall

# set_BC(T)
# set_BC(U)
# set_BC(V)
# set_BC(P)

In [ ]:
def set_NEUMANN(PHI, flag):
    if PHI.neumann is None:
        return
    
    if flag == "pre":
        for j in range(1, NJ):
            if PHI.neumann.get("W") == "neumann": 
                PHI.AP[1][j] -= PHI.AW[1][j]
                PHI.AW[1][j] = 0
            if PHI.neumann.get("E") == "neumann": 
                PHI.AP[NIM][j] -= PHI.AE[NIM][j]
                PHI.AE[NIM][j] = 0
        for i in range(1, NI):
            if PHI.neumann.get("S") == "neumann": 
                PHI.AP[i][1] -= PHI.AS[i][1]
                PHI.AS[i][1] = 0
            if PHI.neumann.get("N") == "neumann": 
                PHI.AP[i][NJM] -= PHI.AN[i][NJM]
                PHI.AN[i][NJM] = 0 
        return

    if flag == "post":
        for j in range(1, NJ):
            if PHI.neumann.get("W") == "neumann": PHI[0][j]  = PHI[1][j]
            if PHI.neumann.get("E") == "neumann": PHI[NI][j] = PHI[NIM][j]
        for i in range(1, NI):
            if PHI.neumann.get("S") == "neumann": PHI[i][0]  = PHI[i][1]
            if PHI.neumann.get("N") == "neumann": PHI[i][NJ] = PHI[i][NJM]
        return

In [ ]:
def divideByZeroWarn(val):
    if np.abs(val)<=1e-30:
        print("WARNING: denominator {} is zero!!".format(val))


def minmod(r):
    return np.fmax(0,np.fmin(1,r))

def get_upwind_id(name, i, j, dir):
    global U,V
    if dir not in ["x", "y"]: AssertionError
    if name=="T":
        if dir=="x" and U[i][j]<0: return (i + 1, j)
        if dir=="y" and V[i][j]<0: return (i, j + 1)
        return (i,j)
    elif name in ["U","V"]:
        if dir=="x" and U[i][j]+U[i+1][j]<0: return (i + 1, j)
        if dir=="y" and V[i][j]+V[i][j+1]<0: return (i, j + 1)
        return (i,j)



def get_faceVal(PHI, i, j, dir, scheme=None):  # MUSCL ALGORITHM
    if dir not in ["x","y"]: AssertionError
    (u_i,u_j)=get_upwind_id(PHI.name,i,j,dir)

    if dir=="x":
        denom=PHI[u_i+1][u_j]-PHI[u_i][u_j]+1e-30
        divideByZeroWarn(denom)
        r=(PHI[u_i][u_j]-PHI[u_i-1][u_j])/denom
    elif dir=="y":
        denom=PHI[u_i][u_j+1]-PHI[u_i][u_j]+1e-30
        divideByZeroWarn(denom)
        r=(PHI[u_i][u_j]-PHI[u_i][u_j-1])/denom
    return PHI[i][j]+0.5*denom*minmod(r)
        

    

def get_RES(PHI):
    global U, V
    global RHS, RES, CORR

    RESMAX = 0
    FFMAX = 0
    FCMAX = 0
    
    # b: Flux Coeff
    if PHI.name=="T":           b=RHO * CP
    elif PHI.name in ["U","v"]: b=RHO
    
    for i in range(1, NI):
        for j in range(1, NJ):
            
            ## advection terms
            # upwind criteria (faced flux)
            if PHI.name=="T":  
                uw=U[i][j]
                ue=U[i+1][j]
                vs=V[i][j]
                vn=V[i+1][j]
            elif PHI.name =="U":
                uw=0.5*(U[i][j]+U[i-1][j])
                ue=0.5*(U[i][j]+U[i+1][j])
                vs=0.5*(V[i-1][j]+V[i][j])
                vn=0.5*(V[i-1][j+1]+V[i][j+1])
            elif PHI.name =="V":
                uw=0.5*(U[i][j-1]+U[i][j])
                ue=0.5*(U[i+1][j-1]+U[i+1][j])
                vs=0.5*(V[i][j]+V[i][j-1])
                vn=0.5*(V[i][j]+V[i][j+1])

            # b: advection coeff
            if PHI.name=="T":           b=RHO * CP
            elif PHI.name in ["U","v"]: b=RHO
            # carrier flux (important)
            Fw = b * RU[i]       * DYP[j] * uw
            Fe = b * RU[i + 1]   * DYP[j] * ue
            Fs = b * RP[i]       * DXP[i] * vs
            Fn = b * RP[i]       * DXP[i] * vn

            neighbors = (
                  PHI.AW[i][j] * PHI[i - 1][j]
                + PHI.AE[i][j] * PHI[i + 1][j]
                + PHI.AS[i][j] * PHI[i][j - 1]
                + PHI.AN[i][j] * PHI[i][j + 1]
            )
            FOU = PHI.AP[i][j] * PHI[i][j] - neighbors
        

            if PHI.name=="T":
                x_advect_1 = Fe * get_faceVal(i, j, "x") - Fw * get_faceVal(i - 1, j, "x")
                y_advect_1 = Fn * get_faceVal(i, j, "y") - Fs * get_faceVal(i, j - 1, "y")

                x_advect_2 = (np.fmax(0, Fe) * PHI[i][j]   + np.fmax(0, -Fe) * PHI[i+1][j]) \
                           - (np.fmax(0, Fw) * PHI[i-1][j] + np.fmax(0, -Fw) * PHI[i][j])

                y_advect_2 = (np.fmax(0, Fn) * PHI[i][j]   + np.fmax(0, -Fn) * PHI[i][j+1]) \
                           - (np.fmax(0, Fs) * PHI[i][j-1] + np.fmax(0, -Fs) * PHI[i][j])

                x_advect = x_advect_1 - x_advect_2
                y_advect = y_advect_1 - y_advect_2
                advect = x_advect + y_advect
            # elif PHI.name in ["U","V"]:
                

            SOU = FOU + advect

            # residual & correction
            RES[i][j] = RHS[i][j] - SOU
            RESMAX = max(RESMAX, np.fabs(RES[i][j] / PHI.AP[i][j]))
            FFMAX = max(FFMAX, np.fabs(PHI[i][j]))
            FCMAX = max(FCMAX, np.fabs(CORR[i][j]))

    FFMAX = max(1.0e-30, FFMAX)
    RESMAX = RESMAX / FFMAX
    FCMAX = FCMAX / FFMAX

    return (RES, RESMAX, FFMAX, FCMAX)
   

### TODO List

1. Upwind criteria for U,V --> flux-based selection
2. Staggered Grid description --> U,V control volume
3. Non-uniform Grid
4. Theory 1, Theory 2
5. Difference of Carrier vs Cargo velocity
   * linked with SIMPLE
     * Carrier: u^n
     * Cargo: u* 

\begin{equation*}
\begin{align*}
LHS_{\text{advection}}
&=(F_e^+ (u_{i,j}) + F_e^- (u_{i+1,j})) \\
&-(F_w^+ (u_{i-1,j}) + F_w^- (u_{i,j})) \\
&+(F_n^+ (u_{i,j}) + F_n^- (u_{i,j+1})) \\
&-(F_s^+ (u_{i,j-1}) + F_s^- (u_{i,j})) \\
\end{align*}
\end{equation*}

Remind that $AP=AW+AE+AS+AN$ so that we finally set LHS as,

\begin{equation*}
LHS_{\text{advection}}=AP\cdot U_{i,j}-(AW\cdot U_{i-1,j}+AE\cdot U_{i+1,j}+AS\cdot U_{i,j-1}+AN\cdot U_{i,j+1})
\end{equation*}

Then,

\begin{equation*}
\begin{align*}
AW&=+F_w^- \\
AE&=-F_e^+ \\
AS&=+F_s^- \\
AN&=-F_n^+ \\
AP&=AW+AE+AS+AN
\end{align*}
\end{equation*}


\begin{equation*}
\begin{align*}
LHS&=LHS_{\text{advection}}+LHS_{\text{diffusion}} \\
&LHS_{\text{advection}} \qquad \text{(second order implicit)} \\
&LHS_{\text{diffusion}} \qquad \text{(first order explicit)} \\
\end{align*}
\end{equation*}

Diffusion terms (also) earned faces, [i+1,i], [i,i-1], [j+1,j], [j,j-1]

In [ ]:
def SOLVE(PHI):
    global U, V, P
    global RHS
    
    if PHI.name not in ["U","V","P","T"]: AssertionError
    RHS=np.zeros((IDIM,JDIM))

    # TDMA coeffs
    for i in range(1, NI):
        for j in range(1, NJ):
            
            # diffusion terms
            ## a: diffusion Coeff
            if PHI.name=="T":           a=K0
            elif PHI.name in ["U","V"]: a=MU0
            ## LHS_diffusion
            AW_diff = a * RU[i] * DYP[j] / DXU[i]
            AE_diff = a * RU[i + 1] * DYP[j] / DXU[i + 1]
            AS_diff = a * RP[i] * DXP[i] / DYV[j]
            AN_diff = a * RP[i] * DXP[i] / DYV[j + 1]

            # advection terms
            ## upwind criteria (faced flux)
            if PHI.name=="T":  
                uw=U[i][j]
                ue=U[i+1][j]
                vs=V[i][j]
                vn=V[i+1][j]
            elif PHI.name =="U":
                uw=0.5*(U[i][j]+U[i-1][j])
                ue=0.5*(U[i][j]+U[i+1][j])
                vs=0.5*(V[i-1][j]+V[i][j])
                vn=0.5*(V[i-1][j+1]+V[i][j+1])
            elif PHI.name =="V":
                uw=0.5*(U[i][j-1]+U[i][j])
                ue=0.5*(U[i+1][j-1]+U[i+1][j])
                vs=0.5*(V[i][j]+V[i][j-1])
                vn=0.5*(V[i][j]+V[i][j+1])

            ## b: advection coeff
            if PHI.name=="T":           b=RHO * CP
            elif PHI.name in ["U","v"]: b=RHO
            ## carrier flux (important)
            Fw = b * RU[i]       * DYP[j] * uw
            Fe = b * RU[i + 1]   * DYP[j] * ue
            Fs = b * RP[i]       * DXP[i] * vs
            Fn = b * RP[i]       * DXP[i] * vn

            # LHS = LHS_diffusion + LHS_advection
            PHI.AW[i][j] = AW_diff + np.fmax(0, +Fw)
            PHI.AE[i][j] = AE_diff + np.fmax(0, -Fe)
            PHI.AS[i][j] = AS_diff + np.fmax(0, +Fs)
            PHI.AN[i][j] = AN_diff + np.fmax(0, -Fn)
            PHI.AP[i][j] = PHI.AW[i][j] + PHI.AE[i][j] + PHI.AS[i][j] + PHI.AN[i][j]
            
            # RHS: regarded as "answer", latest updated
            if PHI.name == "T": 
                RHS[i][j] = QDOT * RP[i] * DXP[i] * DYP[j]
            ## core advantage of STAGGERED-GRID
            elif PHI.name == "U":
                RHS[i][j] = -(P[i][j]-P[i-1][j])*DYP[i][j]*RU[i]
            elif PHI.name == "V":
                RHS[i][j] = -(P[i][j]-P[i][j-1])*DXP[i][j]*RP[j]
                
            
    # boundary conds
    set_NEUMANN(PHI, "pre")

    # earn T_hat (T_numerical)
    for iter in range(ITERMAX):
        (RES, RESMAX, FFMAX, FCMAX) = get_RES()
        if monitor_RES(iter, RESMAX, FCMAX) == 0:
            break

        CORR = LLTDMA(1, NIM, 1, NJM, AW, AE, AS, AN, AP, SOR=RES)
        for i in range(1, NI):
            for j in range(1, NJ):
                PHI[i][j] += PHI.relax * CORR[i][j]

        set_NEUMANN(PHI, "post")

    return PHI.PHI
